In [1]:
!pip install sentencepiece
!pip install -U transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import spacy
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Trainer

# Load SpaCy's NER model for entity extraction
nlp = spacy.load("en_core_web_sm")


def process_clinical_document(document):
    # Extract medical entities using SpaCy's NER model
    doc = nlp(document)
    entities = [(entity.text, entity.label_) for entity in doc.ents]

    # Prompt the user for a query to search within the document
    query = input("Enter a query to search within the document: ")

    # Retrieve relevant sections based on the query
    relevant_sections = []
    for sentence in doc.sents:
        if query in sentence.text.lower():
            relevant_sections.append(sentence.text)

    # Generate a concise and informative summary of the clinical document
    # Load the pretrained BART summarization model
    bart_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")
    bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

    # Prepare input for the summarization model
    inputs = bart_tokenizer(document, return_tensors="pt")

    # Generate the summary
    summary = bart_model.generate(**inputs, max_length=256)
    summary_text = bart_tokenizer.decode(summary[0], skip_special_tokens=True)

    # Print the extracted entities, retrieved sections, and generated summary
    print("Extracted entities:")
    print(entities)

    print("\nRelevant sections:")
    print("\n".join(relevant_sections))

    print("\nSummary:")
    print(summary_text)

In [5]:
# Example usage
clinical_document = """
Patient: Aakash Singh
Age: 23
Diagnosis: Autism Spectrum Disorder (ASD)

Medical History:
- Patient exhibits difficulty with social interaction and communication.
- Patient displays repetitive and restricted behaviors.
- Patient has been receiving behavioral therapy for the past two years.

Current Treatment Plan:
- Continue behavioral therapy sessions.
- Monitor patient's progress and adjust treatment plan as needed.
"""

process_clinical_document(clinical_document)

Enter a query to search within the document: aakash
Extracted entities:
[('Aakash Singh\nAge', 'PERSON'), ('23', 'CARDINAL'), ('the past two years', 'DATE')]

Relevant sections:

Patient: Aakash Singh
Age: 23
Diagnosis: Autism Spectrum Disorder (ASD)

Medical History:
- Patient exhibits difficulty with social interaction and communication.
- Patient displays repetitive and restricted behaviors.
- Patient has been receiving behavioral therapy for the past two years.



Summary:
Patient: Aakash Singh, 25Age: 23Gender: MaleDiagnosis: Autism Spectrum Disorder (ASD)Patient Status: Appears to be stable.Medical History:============- Patient exhibits difficulty with social interaction and communication.============–- Patient displays repetitive and restricted behaviors.======- Patient has been receiving behavioral therapy for the past two years.============-Current Treatment Plan:- Continue behavioral therapy sessions.- Monitor patient's progress and adjust treatment plan as needed.===========

In [7]:
# Example usage
clinical_document_2 = """
A 23-year-old white female presents with complaint of allergies., Allergy / Immunology, Allergic Rhinitis ,"SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.,Lungs:  Clear.,ASSESSMENT:,  Allergic rhinitis.,PLAN:,1.  She will try Zyrtec instead of Allegra again.  Another option will be to use loratadine.  She does not think she has prescription coverage so that might be cheaper.,2.  Samples of Nasonex two sprays in each nostril given for three weeks.  A prescription was written as well.","allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic
"""

process_clinical_document(clinical_document_2)

Enter a query to search within the document: white female
Extracted entities:
[('23-year-old', 'DATE'), ('Allergy / Immunology', 'ORG'), ('Allergic Rhinitis', 'ORG'), ('23-year-old', 'DATE'), ('Seattle', 'GPE'), ('Claritin', 'ORG'), ('Zyrtec', 'GPE'), ('Allegra', 'ORG'), ('last summer', 'DATE'), ('two weeks ago', 'DATE'), ('daily', 'DATE'), ('Ortho Tri-Cyclen', 'PERSON'), ('Allegra', 'PRODUCT'), ('130 pounds', 'QUANTITY'), ('124/78.,HEENT', 'CARDINAL'), ('Nasal', 'PERSON'), ('Clear', 'GPE'), ('Allergic', 'ORG'), ('Zyrtec', 'NORP'), ('Allegra', 'PRODUCT'), ('Nasonex', 'ORG'), ('two', 'CARDINAL'), ('each nostril', 'DATE'), ('three weeks', 'DATE')]

Relevant sections:

A 23-year-old white female presents with complaint of allergies., Allergy / Immunology, Allergic Rhinitis ,"SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  

Summary:
A 23-year-old white female presents with complaint of allergies., Allergy / Immunology, Allergic Rhinitis,ALLERGIES:,SUBJEC